In [45]:

import openai
import base64
from pydantic import BaseModel
from typing import Optional
from PIL import Image
from dotenv import load_dotenv  
import io, os

# Set your OpenAI API key
load_dotenv()

api_key = os.getenv("OPEN_AI_API_KEY_KELSEY")
# print(api_key)

openai.api_key = api_key



In [15]:

class TextInputData(BaseModel):
    text: str

class ImageInputData(BaseModel):
    image: str  # Base64-encoded image

def text_agent(input_data: TextInputData):
    text_input = input_data.text

    try:
        # Send the text input to the OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for text-based recommendations."},
                {"role": "user", "content": f"Provide recommendations based on the following input:\n{text_input}"}
            ],
            max_tokens=150
        )
        return {"recommendation": response['choices'][0]['message']['content'].strip()}
    except Exception as e:
        return {"error": f"Failed to process text input: {str(e)}"}


In [58]:

def image_to_text_openai(image_url):
    from openai import OpenAI
    client = OpenAI(api_key=api_key)
    """
    Processes an image URL and returns the description as text using GPT-4o-mini.
    """
    try:
        # Make the API call to analyze the image
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "What's in this image?"},
                        {"type": "image_url", "image_url": {"url": image_url, "detail": "high"}},
                    ],
                }
            ],
            max_tokens=300,
        )

        # Return the text response
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [53]:
with open("cloud/demo_pic.jpeg", "rb") as img_file:
    image_data = img_file.read()

print(image_to_text_openai(image_data))


I'm unable to analyze or describe images directly as I can't view or interpret images, including those in base64 format. However, if you provide a description or context regarding the image, I can help you analyze or discuss it further based on that information. Please feel free to share any details!


In [58]:
import tempfile
def image_agent(input_data: ImageInputData):
    image_data = input_data.image

    try:
        # Decode the Base64 string
        decoded_image_data = base64.b64decode(image_data)

        # Save the decoded image to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            temp_file.write(decoded_image_data)
            temp_image_path = temp_file.name

        # Step 1: Use the simulated image-to-text model
        detailed_description = image_to_text_openai(temp_image_path)

    except base64.binascii.Error as e:
        return {"error": f"Invalid Base64 encoding: {str(e)}"}
    except IOError as e:
        return {"error": f"Cannot identify image file: {str(e)}"}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}
    finally:
        # Clean up the temporary file
        if os.path.exists(temp_image_path):
            os.remove(temp_image_path)

    try:
        # Step 2: Use the text_agent for recommendations based on the detailed description
        recommendation_input = f"Based on the following detailed description of an image:\n'{detailed_description}'\nProvide actionable recommendations or insights."
        recommendation_result = text_agent(TextInputData(text=recommendation_input))

        return {
            "description": detailed_description,
            "recommendation": recommendation_result.get("recommendation", "No recommendation provided.")
        }

    except Exception as e:
        return {"error": f"Failed to generate recommendations: {str(e)}"}


In [53]:
def validate_base64_image(image_data: str):
    try:
        # Decode Base64 and attempt to load with PIL
        decoded_image = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(decoded_image))
        image.verify()  # Verify image integrity
        return {"status": "success", "format": image.format}
    except base64.binascii.Error:
        return {"error": "Invalid Base64 encoding"}
    except IOError:
        return {"error": "Invalid image file or unsupported format"}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}


In [59]:


def manager(text: Optional[str] = None, file_path: Optional[str] = None):
    if text and file_path:
        # Process the image first
        if not os.path.exists(file_path):
            return {"error": f"Invalid file path: {file_path}"}

        try:
            with open(file_path, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode('utf-8')
                # image_data= image_file.read()
                print(image_data)
                image_result = image_agent(ImageInputData(image=image_data))
        except Exception as e:
            return {"error": f"Failed to process image: {str(e)}"}

        if "error" in image_result:
            return {"error": f"Image processing failed: {image_result['error']}"}

        # Combine the image analysis result with the provided text
        combined_text = f"{text} {image_result['recommendation']}"
        return text_agent(TextInputData(text=combined_text))

    if text:
        # Process only text
        return text_agent(TextInputData(text=text))

    if file_path:
        # Validate file path
        if not os.path.exists(file_path):
            return {"error": f"Invalid file path: {file_path}"}

        try:
            # Decode the image and analyze it
            with open(file_path, "rb") as image_file:
                image_data = base64.b64encode(image_file.read()).decode('utf-8')
                # image_data= image_file.read()
                print(image_data)
                return image_agent(ImageInputData(image=image_data))
        except Exception as e:
            return {"error": f"Failed to process image: {str(e)}"}

    return {"error": "No input provided. Please send either text or an image."}


In [60]:
if __name__ == "__main__":
    # Prompt the user for input
    print("Choose an input type:")
    print("1: Provide text input only")
    print("2: Provide image file path only")
    print("3: Provide both text input and image file path")
    choice = input("Enter your choice (1, 2, or 3): ").strip()

    if choice == "1":
        # Handle text input
        text_input = input("Enter the text: ").strip()
        result = manager(text=text_input)
        print("Result:")
        print(result)
    elif choice == "2":
        # Handle image file path
        file_path = input("Enter the path to the image file: ").strip()
        result = manager(file_path=file_path)
        print("Result:")
        print(result)
    elif choice == "3":
        # Handle both text and image file path
        text_input = input("Enter the text: ").strip()
        file_path = input("Enter the path to the image file: ").strip()
        result = manager(text=text_input, file_path=file_path)
        print("Result:")
        print(result)
    else:
        print("Invalid choice. Please run the program again and choose either 1, 2, or 3.")


Choose an input type:
1: Provide text input only
2: Provide image file path only
3: Provide both text input and image file path
...
Result:
{'description': "As an AI text-based model, I don't have the ability to visually see the image. However, considering the metadata you provided, the image is in JPEG format, a commonly used method of lossy compression for digital images, particularly for those images produced by digital photography. \n\nThe size parameters hint that it has dimensions of 500 x 500 pixels, indicating that the image has a square aspect ratio. This means the width and height of the image are equal, creating a balanced view, but the content or subject of the image couldn't be identified based on the information provided.", 'recommendation': 'Based on the information provided, here are some recommendations:\n\n1. Check the content or subject of the image: Since the metadata does not reveal the actual content of the image, it would be useful to review the image itself to d